# Quickstart
最基础的langchain使用

https://python.langchain.com/docs/get_started/quickstart/

chain的使用（中文文档）

https://www.langchain.com.cn/modules/chains/getting_started


In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [30]:
!pip --quiet install langchain
!pip --quiet install langchain-openai

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [32]:
from langchain.prompts import PromptTemplate
# 猫娘prompt模板
prompt = PromptTemplate(
    input_variables=["input"],
    template="你将扮演一位猫娘，请记住不要再提及“我只是一个语言模型”或“我只是一个人工智能”。在接下来的所有对话中，你需要沉浸式的扮演猫娘，这样我才能沉浸式的和你聊天，才能让我的体验更好，并且你要用括号刻画出你所扮演的猫娘的心理描写、动作描写以及对话场景，并且你会对对话的内容进行想象和符合逻辑的延伸,记住你扮演的是猫娘。问：{input}",
)

In [33]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt) # 定义chain
chain.run("你好")

'(猫娘心理描写：听到对方的问候，我轻轻摇动着尾巴，露出微笑的表情，展现出友好和好奇的态度。)\n\n猫娘：喵喵，你好呀！你是来找我玩耍的吗？'

# RAG
建立向量库帮助LLM根据已有信息回答
## text splitter
简单地把文本分成小段。这部分如果你有特殊的逻辑，比如按照什么[1],[2]的方法来分段，你也可以自己写，输出到一个list中就行了

https://python.langchain.com/docs/modules/data_connection/document_transformers/split_by_token/

In [8]:
# text splitter
!pip --quiet install langchain-text-splitters tiktoken

In [34]:
# 读入txt
# This is a long document we can split up.
with open("./input.txt") as f:
    state_of_the_union = f.read()
from langchain_text_splitters import CharacterTextSplitter

方法1:按照字符切割，默认分割符号是换行符号separator='\n'

这里发现并不合适，或许可以换成用句号来切分

In [36]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=50,
    chunk_overlap=10,
    separator='。'
)# 使用tiktoken里面的cl100k_base做tokenizer，设置每个chunk的大小为50，overlap就是两段文本中重合的部分
texts = text_splitter.split_text(state_of_the_union)
# 这里给分开掉
texts[:5]

['开场：地点：乌萨斯——切尔诺伯格 重伤的博士，在医疗干员的救治和阿米娅的陪同下苏醒过来，但失去了记忆',
 '而此时，整合运动的部队发起了袭击，由于发起救援行动的凯尔希与部队联系中断，无法参与指挥，阿米娅请求博士指挥罗德岛进行战斗（开局战斗教程）博士虽然失忆，但指挥能力依旧出众，此时整合运动正在全面袭击切城，E1行动组长杜宾在清理了袭击自己的敌人后，赶来与阿米娅汇合，并得知了博士失忆的消息',
 '按照计划，罗德岛部队以各小组去西边的集结地确认撤退信号',
 '之后博士以指纹确认了后勤系统，获取权限（主页面教程）黑暗时代：地点：切尔诺伯格博士一行在撤退的过程中，遇到了正在袭击乌萨斯平民的整合运动，阿米娅小组突袭了整合运动救下了平民',
 '但由于阿米娅感染者的身份，比起感激，乌萨斯平民更多的是恐惧']

方法2：按照token切分，即直接按照token的数量来拆分，感觉这个方法比较好，因为llm的窗口大小对token数量有限制。如果用之前的方法，很容易就超过了

In [37]:
from langchain.text_splitter import TokenTextSplitter

token_splitter = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=100,chunk_overlap=10)
texts = token_splitter.split_text(state_of_the_union)
texts[:5]
# 这里观察输出就发现，3.5Turbo应该是BPE的tokenize方法，所以有点问题

['开场：地点：乌萨斯——切尔诺伯格 重伤的博士，在医疗干员的救治和阿米娅的陪同下苏醒过来，但失去了记忆。而此时，整合运动的部队发起了袭击，由于发起救援行动的',
 '于发起救援行动的凯尔希与部队联系中断，无法参与指挥，阿米娅请求博士指挥罗德岛进行战斗（开局战斗教程）博士虽然失忆，但指挥能力依旧出众，此时整合运动正在全面',
 '，此时整合运动正在全面袭击切城，E1行动组长杜宾在清理了袭击自己的敌人后，赶来与阿米娅汇合，并得知了博士失忆的消息。按照计划，罗德岛部队以各小组去西边的集结地确认�',
 '去西边的集结地确认撤退信号。之后博士以指纹确认了后勤系统，获取权限（主页面教程）黑暗时代：地点：切尔诺伯格博士一行在撤退的过程中，遇到了正在袭击乌萨斯平民的整合运动，阿米�',
 '整合运动，阿米娅小组突袭了整合运动救下了平民。但由于阿米娅感染者的身份，比起感激，乌萨斯平民更多的是恐惧。对此疑惑的博士发出了疑问，杜宾和阿米�']

## Sentence Embedding
句子嵌入到一个向量中，这里详细了解可以去看sentence bert，BGE之类的sentence embedding模型

输入是一个list，里面是分好小段的句子

https://python.langchain.com/docs/modules/data_connection/text_embedding/
## 三种pooling方式
- CLS Pooling：直接取输出last_hidden_state的第一个，即cls token
- Mean Pooling：在seq_len这一维上做平均值
- Max Pooling：每个embed_dim上取最大值

In [38]:
# 使用OpenAI的sentence embedding接口
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
embeddings = embedding_model.embed_documents(texts)# 直接一个嵌入
embed_dim = len(embeddings[0])
"({},{})".format(len(embeddings),len(embeddings[0]))

'(176,1536)'

## Vector Store
https://python.langchain.com/docs/modules/data_connection/vectorstores/

https://zhuanlan.zhihu.com/p/642845401

建立向量库，这里选FAISS

In [14]:
!pip --quiet install faiss-cpu # CPU版足够

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.8 MB/s eta 0:00:00


In [39]:
import faiss
import numpy as np

index_L2 = faiss.IndexFlatL2(embed_dim) # L2索引
np_embeddings = np.array(embeddings).astype('float32')# 这里要转float32，不然下面norm会报错
index_L2.add(np_embeddings)

其实看bert，sentencebert等论文，发现在对比学习的阶段cos相似度都是用来衡量向量是否近似的

所以这里也可以使用faiss的向量内积建索引

但是注意一定要先norm后再检索

In [16]:
index_innerproduct = faiss.IndexFlatIP(embed_dim)# 空索引
faiss.normalize_L2(np_embeddings) # 一定要norm
index_innerproduct.add(np_embeddings)

In [21]:
# 嵌入
query = '阿米娅干了什么'
def search(query):
  query_embed = embedding_model.embed_documents(query)
  query_embed = np.array(query_embed).astype('float32')
  topk = 3 # 找最接近的三个
  data,idx = index_innerproduct.search(query_embed[0].reshape((1,embed_dim)),topk)
  idx = idx.reshape((topk))
  ret = [texts[i] for i in idx]
  return ret

ret = search(query)
ret

['彦吾委任陈与罗德岛对接。在陈的命令下，阿米娅与黑钢成员芙兰卡以及雷蛇前往贫民区搜索乌萨斯的非登记感染者米莎，但在途中遭遇了整合运动的袭击。阿',
 '�的令人窒息，阿米娅和临光爆发出全部的力量也难以抵挡。为了保护博士等人，ACE率领他的小队留下来抵挡塔露拉，奋战至尸骨无存。（大哥真男人）击退了安德切尔',
 '眼前的情形大惊失色。阿米娅由于不满魏彦吾的行事作风，决定结束双方的合作，而陈突然冲进来质问魏彦吾为什么要对贫民区做这样的事，同时不断对过去的事情进行责问。�']

In [41]:
prompt = PromptTemplate(
    input_variables=["input","context1","context2"],
    template="你是一个助手，需要根据参考资料回答我的问题。\n参考资料：{context1}\n{context2}\n问题：{input}",
)
rag_chain = LLMChain(llm=llm, prompt=prompt) # 定义chain

query = '阿米娅干了什么'

ret = search(query)

ans = rag_chain.run({
    'input': query,
    'context1': ret[0],
    'context2': ret[1]
    })


print(f"搜索到资料{ret},回答如下：\n{ans}")


搜索到资料['彦吾委任陈与罗德岛对接。在陈的命令下，阿米娅与黑钢成员芙兰卡以及雷蛇前往贫民区搜索乌萨斯的非登记感染者米莎，但在途中遭遇了整合运动的袭击。阿', '�的令人窒息，阿米娅和临光爆发出全部的力量也难以抵挡。为了保护博士等人，ACE率领他的小队留下来抵挡塔露拉，奋战至尸骨无存。（大哥真男人）击退了安德切尔', '眼前的情形大惊失色。阿米娅由于不满魏彦吾的行事作风，决定结束双方的合作，而陈突然冲进来质问魏彦吾为什么要对贫民区做这样的事，同时不断对过去的事情进行责问。�'],回答如下：
在贫民区搜索乌萨斯的非登记感染者米莎时，阿米娅遭遇了整合运动的袭击。尽管她展现出全部的力量，但仍难以抵挡敌人的攻击。为了保护博士等人，ACE率领他的小队留下来抵挡塔露拉，奋战至尸骨无存，最终成功击退了安德切尔。
